<a href="https://colab.research.google.com/github/Aakanksha2518/Female-Diabetes-Prediction-/blob/main/Diab_pred_using_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/content/Diabetes.csv')

In [ ]:
x = df.drop('Outcome', axis =1).values
y = df['Outcome'].values

In [ ]:
def one_hot(y, num_classes):
  m = y.shape[0]
  one_hot_y = np.zeros((m, num_classes))
  one_hot_y[np.arange(m), y] =1
  return one_hot_y


In [ ]:
y_encoded = one_hot(y, 2)

In [ ]:
#splitting the data
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size = 0.2, random_state= 42 )

In [ ]:
#standardising the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [ ]:
#using the softmaa function
def softmax(z):
  #to prevent the model from computing large values like e^1000, which might say inf or NaN, we tend to divide the
  #exponentials by the max exponential, which causes no difference as we are dividing the numerator and denominator both
  exp_z = np.exp(z -np.max(z, axis =1, keepdims=True))
  return exp_z / np.sum(exp_z, axis =1, keepdims=True)




In [ ]:
#model predicting value for each class
def predict_proba(x,w,b):
  return softmax(np.dot(x,w) +b)

In [ ]:
#choosing the max^m probility class, i.e. the final answer
def predict(x, w ,b):
  probs = predict_proba(x, w, b)
  return np.argmax(probs, axis =1 )

In [ ]:
def compute_cost(X, y, W, b):
    m = X.shape[0]
    probs = predict_proba(X, W, b)
    log_probs = np.log(probs + 1e-15)
    loss = -np.sum(y * log_probs) / m
    return loss

In [ ]:
#cost function
#using the cross entropy function to calculate the loss
def compute_gradient(x, y, w, b):
  m = x.shape[0]
  probs = predict_proba(x, w,b)
  dz = probs - y
  dw = np.dot(x.T, dz) / m
  db = np.sum(dz, axis = 0) / m
  return dw, db


In [ ]:
#training loop
def train_softmax(x, y, lr=0.1, epochs=100000):
  n_features = x.shape[1]
  n_classes = y.shape[1]
  w = np.zeros((n_features, n_classes))
  b = np.zeros((n_classes, ))
  losses =[]

  for i in range(epochs):
    dw , db = compute_gradient(x, y, w, b)
    w = w - lr*dw
    b = b - lr*db
    if i % 100 == 0:
      loss = compute_cost(x, y, w, b)
      losses.append(loss)
      print(f"Epoch {i}, Loss {loss:.4f} ")
  return w, b

In [ ]:
#train the model
w_final, b_final = train_softmax(x_train, y_train, lr = 0.1, epochs = 100000)

In [ ]:
#finally using these values of w and b to get the prediction
y_pred = predict(x_test, w_final, b_final)
y_test_labels = np.argmax(y_test, axis =1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")
